<a href="https://colab.research.google.com/github/Jaime44/KUNVENO/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

In [1]:
import sys
import os


# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/PRÁCITCAS EXT/KUNVENO/KUNVENO_DIGITAL/'


pat_dir_checkpoints = path_absolute+'checkpoints'

El código se está ejecutando en Google Colab.
Mounted at /content/drive
Contenido de la carpeta en Google Drive:
['Git_commnads.txt', '.git', 'checkpoints', 'icefall', '.gitmodules', '.gitignore', 'icefall_yes_no_dataset_recipe_with_CPU.ipynb', 'README.md', 'pruned_transducer_statelessx_with_GPU.ipynb']


In [2]:
!ls -la

total 92
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   439 Oct 25 18:36 .gitignore
-rw------- 1 root root    79 Oct 16 19:29 .gitmodules
drwx------ 2 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 55078 Oct 26 19:02 icefall_yes_no_dataset_recipe_with_CPU.ipynb
-rw------- 1 root root 22439 Nov 17 17:14 pruned_transducer_statelessx_with_GPU.ipynb
-rw------- 1 root root    87 Nov  1 08:25 README.md


In [3]:
!pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


### Install PyTorch and torchaudio

In [4]:
import torch
print(torch.__version__)

2.1.0+cu118


Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [5]:
! pip install torchaudio==2.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [6]:
# ! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html
! pip install k2==1.24.3.dev20230718+cuda11.7.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 MB 9.1 MB/s eta 0:00:00


Check that k2 was installed successfully:

In [7]:
! python3 -m k2.version


k2 version: 1.24.3
Build type: Release
Git SHA1: e400fa3b456faf8afe0ee5bfe572946b4921a3db
Git date: Sat Jul 15 04:21:50 2023
Cuda used to build k2: 11.7
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.26.4
GCC version: 9.3.1
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -u

### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [8]:
! pip install lhotse
! pip install git+https://github.com/lhotse-speech/lhotse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.9/731.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 11.5 MB/s eta 0:00:00
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=21d13484bd8ca5ad26b2724dd198ab4575b4a8d717843a64a0e958b611b1d0b7
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-zv0brg3t
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-zv0brg3t
  Resolved https://github.com/lhotse-speech/lhotse to commit 3c0574fdcc39b0ae24ac4c9fdc0f126d16dfebe8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lhotse: filename=lhotse-1.17.0.dev0+git.3c0574f.clean-py3-none-any.whl size=743660 sha256=cb2606106c063c97c1e98b1d961f3f4572092d49a055486b2253e6555fd5a3ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-7n0v95ya/wheels/df/b0/ff/cce0f16868fcdbee2088f3acf9f249dc90117d5f5dd9b6f69d
Successfully built lhotse
  Attempting uninstall: lhotse
    Found existing installation: lhotse 1.17.0
    Uninstalling lhotse-1.17.0:
      Successfully uninstalled lhotse-1.17.0


### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [9]:
! pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


Now install dependencies of `icefall`:

In [10]:
!pip install typing-extensions==3.10.0.2


  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.23 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
arviz 0.15.1 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
chex 0.1.7 requires typing-extensions>=4.2.0; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
flax 0.7.5 requires typing-extensions>=4.2, but you have typing-extensions 3.10.0.2 which is incompatible.
ibis-framework 6.2.0 requires typing-extensions<5,>=4.3.0, but you have typing-extensions 3.10.0.2 which is incompatible.
librosa 0.10.1 requires typing-extensio

In [11]:
! cd icefall && \
  pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4

## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [12]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

Found existing installation: tensorflow 2.14.0
Uninstalling tensorflow-2.14.0:
  Successfully uninstalled tensorflow-2.14.0


In [13]:
# !sudo chmod -R 755 icefall/egs/librispeech/ASR/

In [14]:
!rm -r icefall/egs/librispeech/ASR/shared && \
  sudo ln -s ../../../icefall/shared/ icefall/egs/librispeech/ASR/shared


In [15]:
!rm -r icefall/icefall/transformer_lm/scaling.py && \
ls -la icefall/icefall/transformer_lm/


total 69
-rw------- 1 root root 22349 Oct 12 19:52 attention.py
-rw------- 1 root root  5497 Oct 12 19:52 compute_perplexity.py
-rw------- 1 root root    20 Oct 12 19:52 dataset.py
-rw------- 1 root root 11234 Oct 12 19:52 encoder.py
-rw------- 1 root root  4925 Oct 12 19:52 export.py
-rw------- 1 root root     0 Oct 12 19:52 __init__.py
-rw------- 1 root root  3396 Oct 12 19:52 model.py
drwx------ 2 root root  4096 Oct 12 19:56 __pycache__
-rw------- 1 root root 17243 Oct 12 19:52 train.py


In [16]:
!sudo ln -s ../../egs/librispeech/ASR/pruned_transducer_stateless2/scaling.py icefall/icefall/transformer_lm/scaling.py

In [17]:
# !sudo ln -s ../../../librispeech/ASR/local/prepare_lang_fst.py icefall/egs/yesno/ASR/local/prepare_lang_fst.py && \
# sudo chmod 755 icefall/egs/yesno/ASR/local/prepare_lang_fst.py

In [18]:
!apt-get update && apt-get install dos2unix

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [624 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.0 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,203 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,010 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease

In [19]:
# !dos2unix icefall/egs/yesno/ASR/prepare.sh
# !dos2unix icefall/icefall/shared/parse_options.sh

In [20]:
# !find icefall -type f -print0 | xargs -0 dos2unix

In [21]:
!sudo chmod 755 icefall/egs/librispeech/ASR/local/compute_fbank_librispeech.py

In [22]:
!sudo chmod 755 icefall/egs/librispeech/ASR/prepare.sh

In [ ]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd icefall/egs/librispeech/ASR && \
  ./prepare.sh
  # rm -rf data && \

2023-11-17 17:20:58 (prepare.sh:63:main) dl_dir: /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/librispeech/ASR/download
2023-11-17 17:20:58 (prepare.sh:66:main) Stage -1: Download LM
2023-11-17 17:20:58 (prepare.sh:75:main) Stage 0: Download data
2023-11-17 17:20:59 (prepare.sh:97:main) Stage 1: Prepare LibriSpeech manifest
2023-11-17 17:20:59 (prepare.sh:108:main) Stage 2: Prepare musan manifest
2023-11-17 17:20:59 (prepare.sh:119:main) Stage 3: Compute fbank for librispeech
2023-11-17 17:21:10,322 INFO [compute_fbank_librispeech.py:160] {'bpe_model': None, 'dataset': None, 'perturb_speed': True}
----------------------------------------------------------------------------------------> Manifests: {'dev-clean': {'recordings': RecordingSet(len=2703), 'supervisions': SupervisionSet(len=2703)}, 'dev-other': {'recordings': RecordingSet(len=2864), 'supervisions': SupervisionSet(len=2864)}, 'test-clean': {'recordings': RecordingSet(len=2620), 'supervisions': S

In [ ]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd icefall/egs/librispeech/ASR && \
#   ./prepare.sh --stage 1

In [ ]:
!sudo chmod 755 icefall/egs/librispeech/ASR/pruned_transducer_stateless4/train.py

In [ ]:
!nvcc --version

In [ ]:
# !rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless2/encoder_interface.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/conformer.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decoder.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/joiner.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/model.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/asr_datamodule.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/encoder_interface.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling_converter.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/beam_search.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decode_stream.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/lstmp.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/streaming_beam_search.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/test_model.py && \
# rm -r icefall/egs/aishell/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# rm -r icefall/egs/librispeech/ASR/transducer/asr_datamodule.py && \
# ln -s ../tdnn_lstm_ctc/asr_datamodule.py icefall/egs/librispeech/ASR/transducer/asr_datamodule.py && \
# ln -s ../transducer/asr_datamodule.py icefall/egs/librispeech/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# ln -s ../pruned_transducer_stateless2/optim.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py && \
# ln -s ../pruned_transducer_stateless2/conformer.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/conformer.py && \
# ln -s ../pruned_transducer_stateless2/decoder.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decoder.py && \
# ln -s ../pruned_transducer_stateless2/joiner.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/joiner.py && \
# ln -s ../pruned_transducer_stateless2/model.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/model.py && \
# ln -s ../pruned_transducer_stateless2/asr_datamodule.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/asr_datamodule.py && \
# ln -s ../pruned_transducer_stateless2/encoder_interface.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/encoder_interface.py && \
# ln -s ../pruned_transducer_stateless2/scaling.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling.py && \
# ln -s ../pruned_transducer_stateless3/scaling.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling_converter.py && \
# ln -s ../pruned_transducer_stateless2/beam_search.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/beam_search.py && \
# ln -s ../pruned_transducer_stateless/decode_stream.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decode_stream.py && \
# ln -s ../pruned_transducer_stateless/test_model.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/test_model.py && \
# ln -s ../pruned_transducer_stateless3/lstmp.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/lstmp.py && \
# ln -s ../pruned_transducer_stateless2/streaming_beam_search.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/streaming_beam_search.py && \
# ln -s ../tdnn_lstm_ctc/asr_datamodule.py icefall/egs/aishell/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# ln -s ../transducer_stateless/encoder_interface.py icefall/egs/librispeech/ASR/pruned_transducer_stateless2/encoder_interface.py

In [ ]:
!ls -la icefall/egs/aishell/ASR/pruned_transducer_stateless2

In [ ]:
!ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless2

In [ ]:
!ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless4

In [ ]:
# !export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
#   ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless2 && \
#   ls -la icefall/egs/librispeech/ASR/transducer && \
#   ls -la icefall/egs/aishell/ASR/pruned_transducer_stateless2 && \
#   cd icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
#   ./train.py --help

In [ ]:
! nvidia-smi

In [ ]:
!export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
  export CUDA_VISIBLE_DEVICES="0" && \
  ./train.py --world-size 1

In [ ]:
!export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  export CUDA_VISIBLE_DEVICES="0" && \
  cd icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
  ./train.py \
   --world-size 4 \
   --dynamic-chunk-training 1 \
   --causal-convolution 1 \
   --num-epochs 30 \
   --start-epoch 1 \
   --exp-dir pruned_transducer_stateless4/exp \
   --full-libri 1 \
   --max-duration 300

------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
!echo $CUDA_VISIBLE_DEVICES


-----------------------------------------------------------------------------------------------------------------------------------------------------------

## Training

In [ ]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
# cd icefall/egs/yesno/ASR && \
# ./tdnn/train.py

In [ ]:
# !ls -la


In [ ]:
# !cd icefall/egs/yesno/ASR/tdnn/exp/tensorboard && \
# tensorboard dev upload --logdir . --description "TDNN training for yesno with icefall"

## Decoding

In [ ]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/decode.py

### Show the decoding result

In [ ]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cat tdnn/exp/recogs-test_set.txt

### Show the detailed WER

In [ ]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cat tdnn/exp/errs-test_set.txt

# Pre-trained model

### Download the pre-trained model

In [ ]:
# # ! cd /content/icefall/egs/yesno/ASR && \
# #   mkdir tmp && \
# #   cd tmp && \
# #   git lfs install && \
# #   git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cd tmp &&

In [ ]:
# #! sudo apt-get install git-lfs

In [ ]:
# ! sudo apt-get install tree

In [ ]:
# # ! cd /content/icefall/egs/yesno/ASR && \
# #   mkdir -p tmp && \
# #   tree tmp

# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   tree tmp

In [ ]:
# ! sudo apt-get install sox

In [ ]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [ ]:
# ! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

## Inference with a pre-trained model


### View help information

In [ ]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py --help

### Decode a single sound file

In [ ]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

### Decode multiple sound files

In [ ]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav